<a href="https://colab.research.google.com/github/SaraCorra/Projects_dec2023/blob/main/Digit_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digit recognition with neural networks
### Course on Optimization for Machine Learning - Dr. F. Ballarin
### Master Degree in Data Analytics for Business, Università Cattolica del Sacro Cuore, Milano

In [ ]:
import numpy as np
import pandas as pd
import typing
import plotly.colors
import plotly.graph_objects as go
import plotly.subplots
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
import jax
import jax.numpy as jnp

The *MNIST database* (Modified National Institute of Standards and Technology database) is a database of handwritten digits that is commonly used for testing new algorithms in the field of machine learning. The database collects images of handwritten digits $0, 1, 2, \dots, 9$ by a group of American Census Bureau employees and a group of American high school students. The handwritten digits have been stored as 28x28 pixel grayscale images, with grayscale levels ranging from 0 to 255.

The database has been divided in training (`mnist_train_small.csv`) and testing (`mnist_test.csv`) datasets. Each row of the `csv` file represents an entry of the dataset (i.e., an image of an handwritten digit). The first column stores the value of the digit represented in the image, while each of the remaining $28 \times 28 = 784$ columns stores the grayscale level of a single pixel of the image.

The goal is to set up a digit *recognition* (i.e., *classification* in data analysis), that using a feedforward neural network is able to predict the value of the digit (i.e., the *label*, in data analysis) of the handwritten image based on the pixels of the image (i.e., the *features*, in data analysis).

In [ ]:
# Loaded the csv file using pandas
df_train = pd.read_csv('/content/sample_data/mnist_train_small.csv', header=None)
df_test=pd.read_csv('/content/sample_data/mnist_test.csv', header=None)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y_train=df_train.iloc[:,0]
X_train=df_train.iloc[:,1:]

   * $\boldsymbol{Y}_{\text{train}}$ is a matrix with as many rows as the training dataset and 10 columns, obtained applying one-hot encoding to the first column of the file `mnist_train_small.csv`;
   * $\boldsymbol{X}_{\text{train}}$ is a matrix with as many rows as the training dataset and 784 columns, obtained normalizing between 0 and 1 the remanining columns of the file `mnist_train_small.csv`.

I generate one-hot encoded labels for the response variable by employing the `get_dummies` function.

In [ ]:
Y= pd.get_dummies(y_train)
Y.head()

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0


In [ ]:
X_train.shape

(20000, 784)

To scale values within the interval $[0,1]$, the following rule is employed:

$$
X=\frac{X-X_{min}}{X_{max}-X_{min}+ \epsilon}
$$

A small value is added at the end to avoid division by zero and ensure numerical stability in the normalization process. In this case, the value chosen for the small constant is 1e-13.

In [ ]:
X=(X_train - X_train.min()) / ((X_train.max() - X_train.min())+1e-13)

The following Python function is used to visualize the first 50 images in the training dataset and their corresponding labels. The function `visualize_digits_and_labels` accepts two input arguments `X` and `Y`, corresponding respectively to the matrix of features and the matrix of one-hot encoded labels. The first 50 entries from the training dataset are extracted to construct the appropriate input arguments.

In [ ]:
def visualize_digits_and_labels(X: np.ndarray, Y: np.ndarray) -> None:
    """Visualize digits as images and labels as titles."""
    # Check input arguments
    assert X.shape[0] == Y.shape[0]
    m = X.shape[0]
    assert X.shape[1] == 28 * 28
    assert Y.shape[1] == 10

    # Reverse one-hot encoding to determine the digit number
    digits = np.argwhere(Y == 1)[:, 1]

    # Create a new figure with 10 images per row, and assign titles to each subplot
    ncols = 10
    nrows = int(np.ceil(X.shape[0] / ncols))
    assert nrows < 6, "Do not try to plot more than 50 images"
    fig = plotly.subplots.make_subplots(rows=nrows, cols=ncols, subplot_titles=[str(d) for d in digits])

    # Loop over images and add to subplot
    j = 0
    for row in range(nrows):
        for col in range(ncols):
            if j >= m:
                break

            img_j = X[j].reshape((28, 28))
            fig.add_heatmap(z=img_j, colorscale="gray_r", showscale=False, row=row + 1, col=col + 1)
            j += 1

    fig.update_yaxes(autorange="reversed", constrain="domain", scaleanchor="x", showticklabels=False)
    fig.update_xaxes(constrain="domain", showticklabels=False)
    fig.show()

In [ ]:
X=X.to_numpy()
Y=Y.to_numpy()

In [ ]:
visualize_digits_and_labels(X[:50,:],Y[:50])

The weights and biases are initialized using the Glorot initialization. They're associated to a feedforward neural network with two hidden layers, composed of 50 and 30 neurons respectively.

In [ ]:
np.random.seed(31 + 300)

d = [784,50,30,10]

W_1 = np.random.normal(0, np.sqrt(2 / (d[0] + d[1])), size=(d[1], d[0]))
print("W_1 shape:", W_1.shape)

b_1 = np.zeros((d[1], 1))
print("b_1 shape:", b_1.shape)
# vector with as many vector as nodes in the first hidden layer

W_2 = np.random.normal(0, np.sqrt(2 / (d[1] + d[2])), size=(d[2], d[1]))
print("W_2 shape:", W_2.shape)

b_2 = np.zeros((d[2], 1))
print("b_2 shape:", b_2.shape)

W_3 = np.random.normal(0, np.sqrt(2 / (d[2] + d[3])), size=(d[3], d[2]))
print("W_3 shape:", W_3.shape)

b_3 = np.zeros((d[3], 1))
print("b_3 shape:", b_3.shape)

w_0 = [W_1, b_1, W_2, b_2, W_3, b_3]

W_1 shape: (50, 784)
b_1 shape: (50, 1)
W_2 shape: (30, 50)
b_2 shape: (30, 1)
W_3 shape: (10, 30)
b_3 shape: (10, 1)


The input layer has a shape of (50, 784), with 50 neurons in the first layer and 784 representing the pixels of a single image (i.e., the features). In contrast, the last layer has a shape of (10, 1), reflecting its role in classification, where 10 corresponds to the number of categories.

Below, a Python function that implements a feedforward neural network is implemented. The function takes as inputs the vector `x` of features, and a list `w` collecting all weights and biases. It uses a `tanh` activation function on hidden layer, and a [softmax function](https://en.wikipedia.org/wiki/Softmax_function) on the output layer
$$ \sigma(\boldsymbol{z}) = \frac{1}{\sum_{h = 0}^{9} \exp(z_h)} \begin{bmatrix}\exp(z_0)\\\exp(z_1)\\\dots\\ \exp(z_9)\end{bmatrix}.$$
 The softmax function is chosen to allow interpretation of the outputs as probabilities. Thus (counting neurons starting from 0), if neuron 0 of the output layer has the largest value of all other neurons of the output layer, then the image will be classified as containing the digit 0; similarly, if neuron 1 has the largest value of all, the image will be classified as containing the digit 1, and so on.

In [ ]:
def softmax(x: np.ndarray):
    e_x=jnp.exp(x)
    return e_x/jnp.sum(e_x, axis=0)

In [ ]:
def feedforward_neural_network_regression(x: np.ndarray, w: typing.List[np.ndarray]) -> np.ndarray:
    """
    Evaluate the feedforward neural.

    Parameters
    ----------
    x : 1d or 2d numpy array
        a single feature vector (1d array) or multiple feature vectors (2d array) for which we desire
        a prediction by evaluation of the neural network.
    w : list of weights and biases of the neural network.

    Returns
    -------
    1x10 prediction associated to the feature vector
    """
    (W_1, b_1, W_2, b_2, W_3, b_3) = w
    # this is the list with all the weights and biases.

    # Handle x of different shapes (come back to this after point 3)
    if len(x.shape) == 2:
        x = x.T
    else:
        x = x.reshape(-1, 1)

    # Layer 0 is composed by the input features x
    layer_0 = x

    layer_1 = jnp.tanh(jnp.dot(W_1, layer_0) + b_1)

    layer_2 = jnp.tanh(jnp.dot(W_2, layer_1) + b_2)

    # softmax activation to have values bounded between zero and one
    layer_3 = softmax(jnp.dot(W_3, layer_2) + b_3)

    # Apply the transformation back before returning
    if len(x.shape) == 2:
        return layer_3.T
    else:
        return layer_3.reshape(-1, 1)

When dealing with a classification problem with more than 2 classes, the [cross-entropy loss](https://en.wikipedia.org/wiki/Cross_entropy) is a generalization of the logistic loss, and is defined as follows
$$ \ell(\boldsymbol{y}, \widehat{\boldsymbol{y}}) = - \sum_{h = 0}^{9} y^{(h)} \ \log \widehat{y}^{(h)}, $$
where $\boldsymbol{y} \in \mathbb{R}^{10}$ is the one-hot encoded label associated to an entry with features $\boldsymbol{x} \in \mathbb{R}^{784}$, and $\widehat{\boldsymbol{y}} \in \mathbb{R}^{10}$ is the output of the neural network for the same entry $\boldsymbol{x}$.

Below, the implementation of a function that implements the mini-batch empirical risk associated to the cross-entropy loss is provided. The function  takes as inputs a matrix `X` of features (number of rows equal to the mini-batch size, number of columns equal to 784), a matrix `Y` of the one-hot encoded labels (number of rows equal to the mini-batch size, number of columns equal to 10), a list of the weights and biases `w` to be used in the evaluation of the neural network.

In [ ]:
def empirical_risk_slow(X:np.ndarray, y:np.ndarray,w: typing.List[np.ndarray]) -> float:
    m = X.shape[0]
    if len(y.shape) == 2:
        risk = 0.0
        for j in range(m):
            row_sum = 0.0
            y_hat = feedforward_neural_network_regression(X[j], w)
            for i in range(10):
                row_sum += y[j, i] * np.log(y_hat[0, i])
            risk += (-row_sum)
        return risk / m
    else:
        y_hat = feedforward_neural_network_regression(X, w)
        return -np.sum(y * np.log(y_hat))


This initial implementation of the empirical risk enables the manual accumulation of Cross Entropy for each entry in the dataset, followed by averaging over the number of rows. It distinguishes between cases of multiclassification and binary classification.

The subsequent code introduces a more efficient implementation of the same loss criterion.

In [ ]:
def mini_batch_empirical_risk(X:np.ndarray, y:np.ndarray,w: typing.List[np.ndarray]) -> float:
    """
    Evaluate the empirical risk on the training dataset.
    """
    risk= jnp.mean(jnp.sum(y*jnp.log(feedforward_neural_network_regression(X,w)), axis=1))
    return  -risk

In [ ]:
zeros_slow=empirical_risk_slow(np.zeros(784), Y[1], w_0)
zeros=mini_batch_empirical_risk(np.zeros(784), Y[1], w_0)
one_row_slow=empirical_risk_slow(X[1], Y[1], w_0)
one_row=mini_batch_empirical_risk(X[1], Y[1], w_0)
four_rows_slow=empirical_risk_slow(X[:4,:], Y[:4], w_0)
four_rows=mini_batch_empirical_risk(X[:4,:], Y[:4], w_0)

data={'slow empirical risk': [zeros_slow,one_row_slow,four_rows_slow], 'efficient empirical risk':[zeros, one_row, four_rows]}
df=pd.DataFrame(data)
df = pd.DataFrame(data, index=['x: vector of zeros', 'x: first row', 'x: first four rows'])
df

,slow empirical risk,efficient empirical risk
x: vector of zeros,2.302585,2.3025851
x: first row,2.195546,2.1955464
x: first four rows,2.857309,2.857309


In the table above, we observe that both implementations yield identical results.

The mini-batch stochastic gradient method with Adam diagonal scaling is implemented in a Python function. Such function:
   * takes as input the features and labels in the training dataset, the percentage of training features to use in a mini-batch, the function $f$ to be used for the evaluation of the empirical risk on a mini-batch, its gradient $\nabla f$, the value $\alpha$ of the step length, the value $\zeta$ of the regularization coefficient, the values $\gamma_m$ and $\gamma_s$ of the decay coefficients, the maximum number $E_{\max}$ of allowed epochs, and the initial condition $\boldsymbol{w}_{0}$;
   * returns as outputs the optimization variable iterations $\{\boldsymbol{w}_k\}_k$ and the history of the function values $\{f(\boldsymbol{w}_k)\}_k$.

In [ ]:
grad_mini_batch_empirical_risk = jax.grad(mini_batch_empirical_risk, argnums=2)

In [ ]:
def mini_batch_stochastic_adam(
    X: np.ndarray, Y: np.ndarray, perc: float, f: typing.Callable, grad_f: typing.Callable,
    alpha: float, zeta: float, gamma_m: float, gamma_s:float,
    E_max: float, w_0: typing.List[np.ndarray]
) -> typing.Tuple[np.ndarray, np.ndarray]:
    """

    Parameters
    ----------
    X, Y : np.ndarray
        features and observations of the training dataset.
    perc : float
        percentage of training features to use a mini-batch.
    f, grad_f : Python function
        callable evaluating the cost function and its gradient, respectively.
    alpha : float
        constant step length.
    epsilon : float
        tolerance for the stopping criterion on the error on the norm of the gradient of the cost.
    gamma_m: parameter
    gamma_s: parameter
    zeta: parameter
    E_max : int
        maximum number of allowed epochs.
    w_0 : list of 2d numpy arrays
        initial condition for weights and biases of the neural network.

    Returns
    -------
    list of 2d numpy arrays
        history of the weights and biases of the neural network over the optimization iterations.
    1d numpy array
        history of the empirical risk function values.
    """

    assert X.shape[0] == Y.shape[0]
    m = Y.shape[0]
    m_b = int(perc * m)
    # how big should the batch be

    # Use JAX just-in-time compilation to improve performance
    f = jax.jit(f)
    # able to speed up the time
    grad_f = jax.jit(grad_f)

    # Prepare lists collecting the required outputs over the iterations
    assert isinstance(w_0, list) # I check that w_0 is a list
    all_w = [w_0]
    all_f = [f(X, Y, w_0)]

    # Prepare iteration counter
    k = 0

    s_k = [np.zeros_like(layer) for layer in w_0]
    m_k = [np.zeros_like(layer) for layer in w_0]

    # Use the epoch number as stopping criterion
    while k < E_max * m / m_b:  # the factor m/m_b makes the conversion from iteration to epoch
        w_k = all_w[k]

        # Draw random indices
        J_k = np.random.choice(m, size=m_b, replace=False)

        # Compute the update direction
        # g_k = grad_f(X[J_k], Y[J_k], w_k)
        # I extract the row corresponding to J_k to compute the
        # stochastic gradient

        grad_k=grad_f(X[J_k], Y[J_k], w_k)
        s_k=[gamma_s*s_k[c]+(1-gamma_s)*grad_k[c]**2 for c in range(len(w_k))]
        s_tilde=[s_k[c]/(1-gamma_s**(k+1)) for c in range(len(w_k))]
        m_k=[gamma_m*m_k[c]-(1-gamma_m)*grad_k[c] for c in range(len(w_k))]
        m_tilde=[m_k[c]/(1-gamma_m**(k+1)) for c in range(len(w_k))]  # corrected m_k
        alpha_k=[alpha/(np.sqrt(s_tilde[c])+zeta) for c in range(len(w_k))]

        # Compute w_{k + 1}
        w_k_plus_1 = [
            w_k[c] + alpha_k[c] * m_tilde[c]
            for c in range(len(w_k))
        ]
        # I update W_1,b_1,W_2 ...
        # same update we used to do previously but in a matrix/vector form

        # Update required outputs
        all_w.append(w_k_plus_1)
        all_f.append(f(X, Y, w_k_plus_1))

        # Increment iteration counter
        k += 1

    # Return the history of the optimization variables and costs
    return all_w, np.array(all_f)

In [ ]:
np.random.seed(31 + 700)

all_w_mini_batch_stochastic_adam, all_f_mini_batch_stochastic_adam = mini_batch_stochastic_adam(
X= X, Y=Y, perc= 0.05, f= mini_batch_empirical_risk, grad_f = grad_mini_batch_empirical_risk,
alpha=0.01, zeta=1e-8, gamma_m=0.9, gamma_s=0.999,
E_max=25, w_0=w_0
)

In [ ]:
fig = go.Figure()
fig.add_scatter(x=np.arange(all_f_mini_batch_stochastic_adam.shape[0]), y=np.sqrt(all_f_mini_batch_stochastic_adam))
fig.update_layout(title="History of cross-entropy loss on training set")
fig.update_xaxes(type="log", exponentformat="power")
fig.update_yaxes(type="log", exponentformat="power")
fig.show()

From the plot, it is evident that the Cross-Entropy loss consistently decreases with increments, each increment being smaller than the preceding ones. Towards the end of the process, a slight increase in oscillations becomes noticeable; however, these oscillations do not impede the overall convergence process.

Now, the test dataset $(\boldsymbol{X}_{\text{test}}, \boldsymbol{Y}_{\text{test}})$ is read. The same normalization to the features and one-hot encoding to the labels of the test dataset is applied.

In [ ]:
y_test=df_test.iloc[:,0]
X_test_pd=df_test.iloc[:,1:]
Y_test= pd.get_dummies(y_test)
Y_test.head()

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,1,0,0
1,0,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0


In [ ]:
min_value = np.min(X_test_pd)
max_value = np.max(X_test_pd)

# Scaling the matrix
X_test = (X_test_pd - min_value) / ((max_value - min_value)+1e-13)
X_test.shape

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning:

In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning:

In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'



(10000, 784)

In [ ]:
Y_test=Y_test.to_numpy()
X_test=X_test.to_numpy()

The following Python function is used to visualize the first 50 images in the testing dataset, their corresponding labels and the label which would have been predicted by the neural network (printed as `true vs predicted` above the image). The function `visualize_digits_and_labels_and_predictions` accepts three input arguments `X`, `Y` and `Y_hat`, corresponding respectively to the matrix of features, the matrix of true labels after one-hot encoding, and the matrix output of the neural network evaluation. Extract the first 50 entries from the test dataset and query the neural network with the optimal weights to construct the appropriate input arguments.

In [ ]:
def visualize_digits_and_labels_and_predictions(X: np.ndarray, Y: np.ndarray, Y_hat: np.ndarray) -> None:
    """Visualize digits as images and labels & predictions as titles."""
    # Check input arguments
    assert X.shape[0] == Y.shape[0] == Y_hat.shape[0]
    m = X.shape[0]
    assert X.shape[1] == 28 * 28
    assert Y.shape[1] == 10
    assert Y_hat.shape[1] == 10

    # Reverse one-hot encoding to determine the digit number
    digits = np.argwhere(Y == 1)[:, 1]

    # Determine predicted digit from probability distributions
    digits_hat = np.argmax(Y_hat, axis=1)

    # Create a new figure with 10 images per row, and assign titles to each subplot
    ncols = 10
    nrows = int(np.ceil(X.shape[0] / ncols))
    assert nrows < 6, "Do not try to plot more than 50 images"
    fig = plotly.subplots.make_subplots(
        rows=nrows, cols=ncols,
        subplot_titles=[str(d) + " vs " + str(d_hat) for (d, d_hat) in zip(digits, digits_hat)]
    )

    # Loop over images and add to subplot
    j = 0
    for row in range(nrows):
        for col in range(ncols):
            if j >= m:
                break

            img_j = X[j].reshape((28, 28))
            fig.add_heatmap(z=img_j, colorscale="gray_r", showscale=False, row=row + 1, col=col + 1)
            j += 1

    fig.update_yaxes(autorange="reversed", constrain="domain", scaleanchor="x", showticklabels=False)
    fig.update_xaxes(constrain="domain", showticklabels=False)
    fig.show()

In [ ]:
minimum=np.argmin(all_f_mini_batch_stochastic_adam)
best_w=all_w_mini_batch_stochastic_adam[minimum]

I choose the optimal weights corresponding to the minimal loss.

In [ ]:
neural=feedforward_neural_network_regression(x= X_test[:50], w= best_w)

In [ ]:
max_indices = neural.argmax(axis=1)

# New binary matrix
Y_hat = np.zeros(neural.shape)
for i in range(neural.shape[0]):
    Y_hat[i, max_indices[i]] = 1

In [ ]:
visualize_digits_and_labels_and_predictions(X= X_test[:50,:], Y= Y_test[:50,:], Y_hat= Y_hat)

There is a single misclassified element, corresponding to the digit '5' in the first row.

A $10 \times 10$ confusion matrix, i.e. a matrix which has 10 rows (i.e., one row per each possible value for the true label), 10 columns (i.e., one column per each possible value of the predicted label), and 100 cells (i.e., the cell $(t, p)$ contains the number of test items which had $t$ as true label and $p$ has predicted label) should be used in this case, since we have 10 possible labels. The diagonal cells of the confusion matrix contain the number of test items that were correctly classified, and the ratio between the number of all correctly classified items and the number of test items gives the accuracy. The off-diagonal cells of the confusion matrix contain the number of test items that were misclassified.

In [ ]:
confusion_regression = np.zeros((10, 10))
for (x_j, y_j) in zip(X_test, Y_test):
    Y_hat_j = feedforward_neural_network_regression(x_j, best_w)
    confusion_regression[int(y_j.argmax()), int(Y_hat_j.argmax())] += 1

In [ ]:
confusion_regression = confusion_regression.astype(int)
confusion=pd.DataFrame(confusion_regression)
for i in range(len(confusion)):
    confusion.iat[i, i] = f'[{confusion.iat[i, i]}]'
confusion.index = [f'digit_{i}' for i in confusion.index]
confusion.columns = [f'digit_{col}' for col in confusion.columns]
confusion

,digit_0,digit_1,digit_2,digit_3,digit_4,digit_5,digit_6,digit_7,digit_8,digit_9
digit_0,[955],0,4,3,1,8,3,1,0,5
digit_1,0,[1116],3,4,0,1,3,2,6,0
digit_2,7,3,[983],8,6,1,5,8,9,2
digit_3,0,2,8,[969],2,10,1,6,7,5
digit_4,2,0,2,0,[944],0,7,2,4,21
digit_5,5,2,0,14,4,[842],12,1,8,4
digit_6,11,3,4,3,8,5,[921],0,3,0
digit_7,1,4,14,6,4,1,0,[970],3,25
digit_8,6,2,3,14,7,6,7,3,[921],5
digit_9,5,3,1,9,12,6,1,3,5,[964]


I compute the overall accuracy by generating predictions for the entire test dataset and comparing them with the true labels.

In [ ]:
y_hat_tot=feedforward_neural_network_regression(x= X_test, w= best_w)
max_indices_y = y_hat_tot.argmax(axis=1)

# New binary matrix
Y_hat_tot = np.zeros(y_hat_tot.shape)
for i in range(y_hat_tot.shape[0]):
    Y_hat_tot[i, max_indices_y[i]] = 1

In [ ]:
from sklearn.metrics import accuracy_score
# Y_test contains true lebels whereas Y_hat_tot the predicted ones
accuracy = accuracy_score(Y_test, Y_hat_tot, normalize=True)

print("Final Accuracy on the all test dataset:", accuracy)

Final Accuracy on the all test dataset: 0.9585


Lastly, I determine the indices of the first 50 misclassified entries of the test dataset, and use the `visualize_digits_and_labels_and_predictions` to visualize the misclassified images.

In [ ]:
k=0 # number of misclassified images
i=0 # to have the information of the row
indices=[]
Y_hat_all=np.zeros(10)
for (x_j, y_j) in zip(X_test, Y_test):
    Y_hat_j = feedforward_neural_network_regression(x_j, best_w)
    if np.argmax(Y_hat_j)!=np.argmax(y_j):
        k+=1
        indices.append(i)
        Y_hat_all=np.vstack((Y_hat_all, Y_hat_j))
    i+=1
    if k==50:
        break

Y_hat_all=Y_hat_all[1:,:]

In [ ]:
visualize_digits_and_labels_and_predictions(X= X_test[indices,:], Y= Y_test[indices,:], Y_hat= Y_hat_all)

### Citations and resources

- Lecture Slides: Referred to the course slides for theoretical insights.
- Colab Notebooks: Utilized notebooks from class discussions as a reference specifically for Glorot initialization, the code implementation of feed-forward neural networks and establishing the starting point for mini-batch stochastic Adam optimization.
- ChatGPT: Leveraged ChatGPT for debugging.